# Мобильный оператор: рекомендация тарифов

Оператор мобильной связи хочет построить систему, способную проанализировать поведение клиентов и предложить пользователям новый, более подходящий каждому отдельному клиенту тариф: «Смарт» или «Ультра».  

В нашем распоряжении данные о поведении клиентов с разметкой того, на какой из этих двух тарифов перешел клиент. __Нужно построить модель для задачи классификации, которая выберет подходящий тариф__. Данные уже предобработаны.  

По требованию заказчика ключевая метрика - accuracy. Минимальный порог 0.75, но заказчик будет тем больше доволен, чем выше значение.

__Дополнительным заданием__ заказчика является проверка модели на адекватность.

## 1. Предварительный осмотр данных

По условию задачи мы должны порекомендовать тариф "ультра" или "смарт", то есть создать модель, которая на основе данных о звонках, сообщениях и использовании Интернета определяла бы один из двух подходящих вариантов. Таким образом, мы решаем задачу классификации, поэтому использовать мы будем алгоритмы-классификаторы.

Импортируем все библиотеки и функции, которые могут понадобиться нам для работы.

In [1]:
import sklearn
import joblib
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

Импортируем файл и изучим его.

In [2]:
df = pd.read_csv('...')
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Поскольку наша задача - на основе данных о звонках, сообщениях и использовании Интернета порекомендовать тариф, столбец is_ultra будет целевым, а остальные - признаками, на основе которых будет прогнозироваться значение в is_ultra.  
Несмотря на то, что тип данных в is_ultra числовой, по сути это булева переменная, принимающая значение 1, если тариф - "ультра", и 0, если тариф - "смарт".

In [30]:
df['is_ultra'].unique()

array([0, 1])

Проверим, как распределены значения о тарифах в общем наборе данных.

In [4]:
len(df.query('is_ultra == 1')) / len(df)

0.30647168637212197

Распределение по тарифам не одинаковое, тарифом "ультра" пользуются 30% пользователей из набора. Будем иметь это в виду при разбиении на обучающую, валидационную и тестовую выборки: стратифицируем разбиение. 

## 2. Создание выборок

Присвоим переменной целое число и будем использовать переменную в качестве зерна для параметра random_state в моделях. Это позволит минимизировать шанс ошибок в случаях, когда нужно будет создать две модели на основании одинаковых алгоритмов с одинаковыми гиперпараметрами, например, когда мы определимся с алгоритмом. Кроме того, это упростит и ускорит работу.

In [5]:
random = 12345 

Разобьем набор данных на обучающий, валидационный и тестовый наборы данных в пропорции 60/20/20 соответственно и проверим пропорцию значений в целевом признаке is_ultra в каждом наборе.

In [6]:
df_train, df_valid_test = train_test_split(df, test_size = 0.4, shuffle = True, stratify = df['is_ultra'], random_state = random)
df_valid, df_test = train_test_split(df_valid_test, test_size = 0.5, shuffle = True, stratify = df_valid_test['is_ultra'], random_state = random)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928 entries, 2294 to 2134
Data columns (total 5 columns):
calls       1928 non-null float64
minutes     1928 non-null float64
messages    1928 non-null float64
mb_used     1928 non-null float64
is_ultra    1928 non-null int64
dtypes: float64(4), int64(1)
memory usage: 90.4 KB


In [7]:
len(df_train.query('is_ultra == 1')) / len(df_train)

0.3065352697095436

In [8]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 78 to 2434
Data columns (total 5 columns):
calls       643 non-null float64
minutes     643 non-null float64
messages    643 non-null float64
mb_used     643 non-null float64
is_ultra    643 non-null int64
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


In [9]:
len(df_valid.query('is_ultra == 1')) / len(df_valid)

0.30637636080870917

In [10]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 3113 to 691
Data columns (total 5 columns):
calls       643 non-null float64
minutes     643 non-null float64
messages    643 non-null float64
mb_used     643 non-null float64
is_ultra    643 non-null int64
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


In [11]:
len(df_test.query('is_ultra == 1')) / len(df_test)

0.30637636080870917

Набор данных разбит на обучающую, валидационную и тестовую части, пропорция значений 1 и 0 целевого признака в каждой из них соответствует пропорции значений в общем наборе данных.

In [12]:
features_train = df_train.drop('is_ultra', axis = 1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop('is_ultra', axis = 1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop('is_ultra', axis = 1)
target_test = df_test['is_ultra']

## 3. Подбор модели

Рассмотрим модели, построенные с помощью дерева решений, случайного леса и логистической регрессии с разными гиперпараметрами.  
Переберем разные сочетания гиперпараметров и посчитаем для каждого сочетания метрики качества.  
Метрик качества у нас будет две: метод score (accuracy_score для дерева решений, значения те же самые, именно ее по заданию нужно довести до порогового значения, и f1_score, при этом со средневзвешенной средней оценкой. Она позволяет чуть более точно определить правильность предсказания для каждого класса отдельно, но при этом благодаря весам класса по количеству значений в выборке не исказится из-за не одинакового распределения значений целевого признака, которые в наших наборах данных распределены в пропорции 30/70.  

### Дерево решений

Рассмотрим дерево рещений с разными сочетаниями значений гиперпараметров. Напишем цикл, перебирающий разные сочетания и рассчитывающий для каждого метрики качества.  
Чтобы перебор не занимал излишне много времени, переберем не каждое, а каждое второе значение гиперпараметров.

In [48]:
tree_depth_col = []
tree_min_leaf_col = []
tree_min_split_col = []
tree_acc_col = []
tree_f1_col = []

for depth in range(2, 21, 2):
    for min_leaf in range(1, 10, 2):
        for min_split in range(2,10,2):
            model_tree = DecisionTreeClassifier(max_depth = depth, min_samples_split = min_split, min_samples_leaf = min_leaf, random_state = random)
            model_tree.fit(features_train, target_train)
            predictions_tree = model_tree.predict(features_valid)
            acc_score_tree = accuracy_score(target_valid, predictions_tree)
            f1_score_tree = f1_score(target_valid, predictions_tree, average = 'weighted')
            tree_depth_col.append(depth)
            tree_min_split_col.append(min_split)
            tree_min_leaf_col.append(min_leaf)
            tree_acc_col.append(acc_score_tree)
            tree_f1_col.append(f1_score_tree)
        
tree_hyperparameters_dict = {'max_depth': tree_depth_col, 'min_samples_split': tree_min_split_col, 'min_samples_leaf': tree_min_leaf_col, 'accuracy_score': tree_acc_col, 'f1_score':tree_f1_col}
tree_hyperparameters = pd.DataFrame(data = tree_hyperparameters_dict)
tree_hyperparameters

,max_depth,min_samples_split,min_samples_leaf,accuracy_score,f1_score
0,2,2,1,0.772939,0.757300
1,2,4,1,0.772939,0.757300
2,2,6,1,0.772939,0.757300
3,2,8,1,0.772939,0.757300
4,2,2,3,0.772939,0.757300
...,...,...,...,...,...
195,20,8,7,0.737170,0.731362
196,20,2,9,0.752722,0.748605
197,20,4,9,0.752722,0.748605
198,20,6,9,0.752722,0.748605


Выберем модели с лучшими значениями метрик и взглянем на указанные в них гиперпараметры.   

In [50]:
tree_hyperparameters.query('accuracy_score == accuracy_score.max()')

,max_depth,min_samples_split,min_samples_leaf,accuracy_score,f1_score
67,8,8,3,0.804044,0.791601


In [43]:
tree_hyperparameters.query('f1_score == f1_score.max()')

,max_depth,min_samples_split,min_samples_leaf,score,accuracy_score,f1_score
454,8,8,3,0.804044,0.804044,0.791601


Обе метрики указывают на одинаквое сочетание значений гиперпараметров. В принципе, полученного на валидационной выборке значения accuracy достаточно, чтобы попробовать провериться на тестовой. Но дерево обучения склонно к переобучению, и показатели accuracy модели на этом алгоритме на тестовой выборке могут оказаться ниже, чем у моделей, обученных с помощью случайного леса или логистической регрессии.  
Поэтому рассмотрим другие алгоритмы, если результаты метрик качества будут сопоставимы, лучше будет отдать предпочтение им.

### Случайный лес

Переберем сочетания гиперпараметров для случайного леса. Перебор может занять много времени, поскольку количество рассматриваемых гиперпараметров увеличивается: появляется количество деревьев. Шаг для него сделаем больше, также изменим количество рассматриваемых решений для других параметров, чтобы у нас появился шанс дождаться перебора сочетаний.

In [58]:
forest_depth_col = []
forest_estim_col = []
forest_min_leaf_col = []
forest_min_split_col = []
forest_score_col = []
forest_f1_col = []

for depth in range(2,16,2):
    for min_leaf in range(1, 6, 2):
        for min_split in range(2,8,2):
            for estim in range(10, 50, 10):
                model_forest = RandomForestClassifier(n_estimators = estim, max_depth = depth, min_samples_split = min_split, min_samples_leaf = min_leaf, random_state = random)
                model_forest.fit(features_train, target_train)
                predictions_forest = model_forest.predict(features_valid)
                score_forest = model_forest.score(features_valid, target_valid)
                f1_forest = f1_score(target_valid, predictions_forest, average = 'weighted')
                forest_depth_col.append(depth)
                forest_min_leaf_col.append(min_leaf)
                forest_min_split_col.append(min_split)
                forest_estim_col.append(estim)
                forest_score_col.append(score_forest)
                forest_f1_col.append(f1_forest)
        
        
forest_hyperparameters_dict = {'max_depth': forest_depth_col, 'n_estimators': forest_estim_col, 'min_samples_split': forest_min_split_col, 'min_samples_leaf': forest_min_leaf_col, 'score': forest_score_col, 'f1_score':forest_f1_col}
forest_hyperparameters = pd.DataFrame(data = forest_hyperparameters_dict)
forest_hyperparameters

,max_depth,n_estimators,min_samples_split,min_samples_leaf,score,f1_score
0,2,10,2,1,0.776050,0.743149
1,2,20,2,1,0.785381,0.762724
2,2,30,2,1,0.793157,0.774360
3,2,40,2,1,0.793157,0.774360
4,2,10,4,1,0.776050,0.743149
...,...,...,...,...,...,...
247,14,40,4,5,0.810264,0.800151
248,14,10,6,5,0.791602,0.777248
249,14,20,6,5,0.805599,0.793511
250,14,30,6,5,0.800933,0.790806


Определим модели с лучшей метрикой score.

In [60]:
forest_hyperparameters.query('score == score.max()')

,max_depth,n_estimators,min_samples_split,min_samples_leaf,score,f1_score
110,8,30,2,1,0.819596,0.808613
115,8,40,4,1,0.819596,0.807170


Примечательно, что в результате перебора у нас появилось две модели с разными гиперпараметрами, но одинаковыми показателями качества. Это можно было бы объяснить тем, что некоторые гиперпараметры задают не точное количество глубины или объектов в листьях, а максимальное, в то время как соответствующие параметры у моделей могут совпадать.  
Однако в этом случае это не так, поскольку различаются значения в гиперпараметре n_estimators, задающее точное количество деревьев в лесу.  
В любом случае, нам повезло, что мы рассматривали не единственную метрику качесва. Различающиеся значения f1_score свидетельствуют о том, что, может, общее количество правильных ответов в моделях и одинаковое, в одном из случаев это количество распределено по классам более пропорционально.  

In [61]:
forest_hyperparameters.query('f1_score == f1_score.max()')

,max_depth,n_estimators,min_samples_split,min_samples_leaf,score,f1_score
110,8,30,2,1,0.819596,0.808613


Случайный лес более устойчив к переобучению, чем дерево решений, поэтому предпочтение отдаем ему. Тем более, показатели качества в его случае выше.

### Логистическая регрессия

Логистическая регрессия работает быстрее, чем случайный лес, поэтому при одинаковой точности предпочтение стоит отдать ей, хотя если точность будет ниже, то выберем мы случайный лес, так как заказчик поставил пороговое значение по точности, а не по быстродействию. 

In [63]:
logreg = LogisticRegression(random_state = random)
logreg.fit(features_train, target_train)
predictions_logreg = logreg.predict(features_valid)
score_logreg = logreg.score(features_valid, target_valid)
f1_score_logreg = f1_score(target_valid, predictions_logreg, average = 'weighted')
print('score:', score_logreg)
print('f1_score:', f1_score_logreg)

score: 0.71850699844479
f1_score: 0.644571277664623


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Показатели качества на валидационной выборке гораздо ниже, чем у предыдущих алгоритмов, поэтому оставляем свой предыдущий выбор.

## 4. Тестирование модели

Проверим модель на тестовой выборке.

In [66]:
best_depth = int(forest_hyperparameters.loc[forest_hyperparameters['f1_score'] == forest_hyperparameters['f1_score'].max(), 
                                            'max_depth'])
best_estim = int(forest_hyperparameters.loc[forest_hyperparameters['f1_score'] == forest_hyperparameters['f1_score'].max(), 
                                            'n_estimators'])
best_split =int(forest_hyperparameters.loc[forest_hyperparameters['f1_score'] == forest_hyperparameters['f1_score'].max(), 
                                           'min_samples_split'])
best_leaf =int(forest_hyperparameters.loc[forest_hyperparameters['f1_score'] == forest_hyperparameters['f1_score'].max(), 
                                          'min_samples_leaf'])

In [68]:

best_model = RandomForestClassifier(max_depth = best_depth, 
                                    n_estimators = best_estim, 
                                    min_samples_split = best_split, 
                                    min_samples_leaf = best_leaf, 
                                    random_state = random)
best_model.fit(features_train, target_train)
best_predictions = best_model.predict(features_test)
best_score = best_model.score(features_test, target_test)
best_f1_score = f1_score(target_test, best_predictions, average = 'weighted')

best_score

0.8055987558320373

In [69]:
best_f1_score

0.7968549064971004

Показатели качества выбранной модели на тестовой выборке переступили пороговое значение, значит, задача-минимум выполнена. Однако заказчик просит довести количество правильных ответов до максимального. Мы можем это сделать, если обучим модель не на первоначально выбранном обучающем наборе, а на объединении первоначальных обучающих и валидационных выборок. Посмотрим, улучшит ли это результат. 

In [70]:
df_train_final = df_train.append(df_valid, ignore_index = True)
features_train_final = df_train_final.drop('is_ultra', axis = 1)
target_train_final = df_train_final['is_ultra']

best_model_final = best_model.fit(features_train_final, target_train_final)

best_predictions_final = best_model_final.predict(features_test)
best_score_final = best_model_final.score(features_test, target_test)
best_f1_score_final = f1_score(target_test, best_predictions_final, average = 'weighted')

best_score_final

0.8289269051321928

In [71]:
best_f1_score_final

0.8176081973240633

Результат улучшен, модель, рекомендующая тарифы подобрана.

## 5. (бонус) Проверка модели на адекватность

Создадим модель, которая будет генерировать случайные предсказания с помощью Dummy Classifier.

In [99]:
from sklearn.dummy import DummyClassifier
random_model = DummyClassifier()
random_model.fit(features_train_final, target_train_final)
random_model.score(features_test, target_test)


0.583203732503888

In [100]:
random_model.score(features_test, target_test)

0.5769828926905132

In [101]:
random_model.score(features_test, target_test)

0.5583203732503889

In [102]:
random_model.score(features_test, target_test)

0.536547433903577

In [103]:
random_model.score(features_test, target_test)

0.578538102643857

In [104]:
random_model.score(features_test, target_test)

0.5412130637636081

In [105]:
random_model.score(features_test, target_test)

0.5396578538102644

In [106]:
random_model.score(features_test, target_test)

0.5956454121306376

Предположим, что значения accuracy больше 0.5, поскольку стратегия случайного прогнозирования по умолчанию stratified, то есть дает случайные прогнозы не 50/50, а в соответствии с пропорцией классов в выборке, поэтому accuracy рассчитывается по формуле 0.3 * 0.3 + 0.7 * 0.7 = 0.58, а не 0.5 * 0.3 + 0.5 * 0.7 = 0.5.  
Впрочем, это даже еще очевиднее демонстрирует, что тест на вменяемость пройден: метрики качества нашей модели выше каждого из этих значений.    

In [89]:
totally_random_model = DummyClassifier(strategy = 'uniform')
totally_random_model.fit(features_train_final, target_train_final)
totally_random_model.score(features_test, target_test)

0.49922239502332816

In [90]:
totally_random_model.score(features_test, target_test)

0.5023328149300156

In [91]:
totally_random_model.score(features_test, target_test)

0.5147744945567652

Более стандартный тест на вменяемость с абсолютно случайными прогнозами 50/50 тоже пройден.